In [1]:
import json
import os
import pandas as pd
from pandas.io.json import json_normalize
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from typing import List

# Setup sql
engine = create_engine("sqlite:///classification_final.db")

session = sessionmaker()
session.configure(bind=engine)
s = session()

dirname = "../congresstweets/data"

# First create tweets table of unique tweets

In [2]:
engine.execute(
    "CREATE TABLE IF NOT EXISTS tweets(id TEXT PRIMARY KEY, text TEXT, screen_name TEXT, user_id TEXT, label INT, is_official INT)"
)

# Load the metadata

In [3]:
def extract_legis_metadata(fn: str) -> pd.DataFrame:
    with open(fn, "r") as f:
        dat = json.load(f)
    df = json_normalize(dat)

    # fix ridiculous nested dict/list/idct
    # "terms" column contains dicts, break em out!
    # we are taking the LAST (MOST RECENT) term for everyone
    terms = pd.DataFrame([i[-1] for i in df.terms.tolist()])
    terms = terms[["type", "state", "party"]]

    df = pd.concat([df, terms], axis=1)[["id.bioguide", "type", "state", "party"]]
    df.columns = ["id_bioguide", "type", "state", "party"]

    return df

In [4]:
# get social media handle - legislator mapping
with open("legislators-social-media.json", "r") as f:
    sm_data = json.load(f)
    
legislator_sm_df = json_normalize(sm_data)[
    ["id.bioguide", "social.twitter_id", "social.twitter"]
]
legislator_sm_df.columns = ["id_bioguide", "twitter_id", "twitter_screenname"]

# needs to be str because the numbers are too large
legislator_sm_df["twitter_id"] = legislator_sm_df["twitter_id"].astype(str)

# need to lowercase for matching
legislator_sm_df["twitter_screenname"] = legislator_sm_df[
    "twitter_screenname"
].str.lower()

In [5]:
legislator_sm_df.head()

,id_bioguide,twitter_id,twitter_screenname
0,R000600,3026622545,repamata
1,Y000064,234128524,sentoddyoung
2,E000295,2856787757,senjoniernst
3,T000476,2964174789,senthomtillis
4,Y000062,384913290,repjohnyarmuth


In [6]:
legislator_sm_df.shape

(532, 3)

In [7]:
legislator_sm_df[legislator_sm_df.isnull().any(axis=1)]

,id_bioguide,twitter_id,twitter_screenname
209,C001049,nan,NaN
239,A000367,nan,NaN
310,K000384,nan,NaN
412,C001108,nan,NaN
431,G000584,nan,NaN


In [8]:
# get legislator - party mapping
current_legis = extract_legis_metadata("legislators-current.json")
historical_legis = extract_legis_metadata(
    "legislators-historical.json"
)

In [9]:
current_legis.tail()

,id_bioguide,type,state,party
530,G000592,rep,ME,Democrat
531,K000395,rep,PA,Republican
532,B001311,rep,NC,Republican
533,M001210,rep,NC,Republican
534,L000594,sen,GA,Republican


In [10]:
historical_legis.tail()

,id_bioguide,type,state,party
11977,C000984,rep,MD,Democrat
11978,H001087,rep,CA,Democrat
11979,I000055,sen,GA,Republican
11980,H001048,rep,CA,Republican
11981,M001187,rep,NC,Republican


In [11]:
current_legis[current_legis.isnull().any(axis=1)] # no nulls here

,id_bioguide,type,state,party


In [12]:
historical_legis[historical_legis.isnull().any(axis=1)].shape

(234, 4)

In [13]:
# is there overlap?

set(current_legis.id_bioguide.unique().tolist()).intersection(
set(historical_legis.id_bioguide.unique().tolist())
)

set()

In [14]:
# combine legislator metadata
all_legislators_metadata_df = pd.concat([current_legis, historical_legis])

In [15]:
# join in order to add state/party metadata to social media df

combined_metadata = legislator_sm_df.merge(
    all_legislators_metadata_df,
    how="left",
    left_on="id_bioguide",
    right_on="id_bioguide",
)


In [16]:
combined_metadata.shape

(532, 6)

In [17]:
combined_metadata.tail()

,id_bioguide,twitter_id,twitter_screenname,type,state,party
527,H001089,1080960924687704064,senhawleypress,sen,MO,Republican
528,V000133,1083469084648505344,congressmanjvd,rep,NJ,Republican
529,K000395,1136060761422405633,repfredkeller,rep,PA,Republican
530,S001204,346509049,guamcongressman,rep,GU,Democrat
531,L000594,1200451909406121984,senatorloeffler,sen,GA,Republican


In [18]:
# where do we have missing data?
combined_metadata[combined_metadata.isnull().any(axis=1)]

,id_bioguide,twitter_id,twitter_screenname,type,state,party
209,C001049,nan,NaN,rep,MO,Democrat
239,A000367,nan,NaN,rep,MI,Independent
310,K000384,nan,NaN,sen,VA,Democrat
412,C001108,nan,NaN,rep,KY,Republican
431,G000584,nan,NaN,rep,MT,Republican


In [19]:
combined_metadata[~combined_metadata.isnull().any(axis=1)].shape

(527, 6)

In [20]:
print(f"combined_metadata before dropping NAs {combined_metadata.shape}")

combined_metadata.dropna(inplace=True)  # drop anyone with incomplete metadata

print(f"combined_metadata after dropping NAs {combined_metadata.shape}")


combined_metadata before dropping NAs (532, 6)
combined_metadata after dropping NAs (527, 6)


In [21]:
combined_metadata = combined_metadata[["twitter_screenname", "type", "state", "party"]]
combined_metadata["is_official"] = 1

In [22]:
combined_metadata.head()

,twitter_screenname,type,state,party,is_official
0,repamata,rep,AS,Republican,1
1,sentoddyoung,sen,IN,Republican,1
2,senjoniernst,sen,IA,Republican,1
3,senthomtillis,sen,NC,Republican,1
4,repjohnyarmuth,rep,KY,Democrat,1


# Load the manually coded data

In [23]:
manually_coded = pd.read_csv("missing_metadata_filled.csv")

In [24]:
# add the is_official column and deal with Independents and Libertarians

manually_coded = manually_coded[~manually_coded.party.isnull()]
manually_coded["is_official"] = 0
manually_coded.replace({"r": "Republican", 
                        "d": "Democrat", 
                        "i": "Independent", 
                        "L": "Libertarian"}, 
                       inplace=True)
manually_coded = manually_coded.rename(columns={"screen_name": "twitter_screenname"})

In [25]:
manually_coded.head()

,twitter_screenname,count,type,state,party,is_official
0,auctnr1,54103,rep,MO,Republican,0
1,hispaniccaucus,27298,NaN,NaN,Democrat,0
2,chiproytx,22404,rep,TX,Republican,0
3,lacyclaymo1,21319,rep,MO,Democrat,0
4,repcloakroom,16333,NaN,NaN,Republican,0


# Combine manually coded and official accounts

In [26]:
all_metadata = pd.concat([manually_coded, combined_metadata])

all_metadata.shape

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


(659, 6)

In [27]:
all_metadata.party.value_counts()

Democrat       365
Republican     290
Independent      3
Libertarian      1
Name: party, dtype: int64

In [28]:
# code Republicans/Libertarians --> 1, Democrats/Independents --> 0
def encode_party(row):
    if row.party in  ["Republican", "Libertarian"]:
        return 1
    else:
        return 0

all_metadata['label'] = all_metadata.apply(lambda row: encode_party(row), axis=1)

In [29]:
all_metadata[all_metadata["party"] == "Independent"]

,count,is_official,party,state,twitter_screenname,type,label
19,8602.0,0,Independent,VT,berniesanders,sen,0
63,NaN,1,Independent,VT,sensanders,sen,0
244,NaN,1,Independent,ME,senangusking,sen,0


In [30]:
all_metadata[all_metadata["party"] == "Libertarian"]

,count,is_official,party,state,twitter_screenname,type,label
71,4217.0,0,Libertarian,MI,justinamash,rep,1


# Finally, write the tweets table with the metadata

In [31]:
# tweets table
for fname in [f for f in os.listdir(dirname) if f.endswith("json")]:
    temp_df = (
        pd.read_json(os.path.join(dirname, fname))
        .dropna()
        .replace("\n", " ", regex=True)
    )
    temp_df["screen_name_lower"] = temp_df["screen_name"].str.lower()
    
    df = temp_df[["id", "text", "screen_name", "user_id", "screen_name_lower"]].merge(
        all_metadata,
        how="inner",
        left_on = "screen_name_lower",
        right_on = "twitter_screenname"
    )

    df[["id", "text", "screen_name", "user_id", "label", "is_official"]].to_sql(
        "temp_table",
        con=engine,
        index=False,
        index_label="id",
        if_exists="replace",
        chunksize=1000,
    )

    del temp_df
    del df
    
    insert_into_sql = "INSERT OR IGNORE INTO tweets SELECT * FROM temp_table"
    engine.execute(insert_into_sql)

    
engine.execute("DROP TABLE IF EXISTS temp_table")

s.commit()

In [32]:
# the twitter user id is only in the tweets data
# need to pull it out to make our users table
twitter_user_ids = pd.read_sql_query(sql = "SELECT screen_name, user_id FROM tweets GROUP BY screen_name",
                 con = engine)
twitter_user_ids["screen_name_lower"] = twitter_user_ids["screen_name"].str.lower()

In [33]:
all_metadata = all_metadata.merge(twitter_user_ids,
                  how="left",
                  left_on="twitter_screenname",
                  right_on="screen_name_lower")

all_metadata.drop(["count", "screen_name_lower", "twitter_screenname"], 
                  axis = 1,
                  inplace = True)

In [34]:
all_metadata.head()

,is_official,party,state,type,label,screen_name,user_id
0,0,Republican,MO,rep,1,auctnr1,21572351
1,0,Democrat,NaN,NaN,0,HispanicCaucus,33530012
2,0,Republican,TX,rep,1,chiproytx,1257667158
3,0,Democrat,MO,rep,0,LacyClayMO1,584912320
4,0,Republican,NaN,NaN,1,RepCloakroom,1137600571


In [35]:
# write the users table
engine.execute(
    "CREATE TABLE IF NOT EXISTS users(user_id TEXT PRIMARY KEY, screen_name TEXT, state TEXT, type TEXT, party TEXT, label INT, is_official TEXT)"
)

In [37]:
all_metadata.to_sql("users",
                    con=engine,
                   index=False,
                   index_label="user_id",
                   if_exists="replace")